# **Import các thư viện**

In [1]:
import pandas as pd
import numpy as np
import re

- Đọc file csv vào bảng df

In [ ]:
df = pd.read_csv('./Datasets/car_sales_data.csv')

df.head()

,Manufacturer,Model,Engine size,Fuel type,Year of manufacture,Mileage,Price
0,Ford,Fiesta,1.0,Petrol,2002,127300,3074
1,Porsche,718 Cayman,4.0,Petrol,2016,57850,49704
2,Ford,Mondeo,1.6,Diesel,2014,39190,24072
3,Toyota,RAV4,1.8,Hybrid,1988,210814,1705
4,VW,Polo,1.0,Petrol,2006,127869,4101


# **Kiểm tra thông tin dữ liệu**

- Xem kiểu dữ liệu của từng cột thuộc tính

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Manufacturer         50000 non-null  object 
 1   Model                50000 non-null  object 
 2   Engine size          50000 non-null  float64
 3   Fuel type            50000 non-null  object 
 4   Year of manufacture  50000 non-null  int64  
 5   Mileage              50000 non-null  int64  
 6   Price                50000 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 2.7+ MB


- Kiểm tra có dữ liệu còn thiếu hay không

In [4]:
df.isna().sum()

Manufacturer           0
Model                  0
Engine size            0
Fuel type              0
Year of manufacture    0
Mileage                0
Price                  0
dtype: int64

# **Làm sạch dữ liệu**

- Simplify tên các cột

In [5]:
df.rename(columns={
    "Manufacturer": "brand",
    "Model": "model",
    "Engine size": "engine_size",
    "Fuel type": "fuel_type",
    "Year of manufacture": "year",
    "Mileage": "mileage_km",
    "Price": "price_usd",
}, inplace=True)

df

,brand,model,engine_size,fuel_type,year,mileage_km,price_usd
0,Ford,Fiesta,1.0,Petrol,2002,127300,3074
1,Porsche,718 Cayman,4.0,Petrol,2016,57850,49704
2,Ford,Mondeo,1.6,Diesel,2014,39190,24072
3,Toyota,RAV4,1.8,Hybrid,1988,210814,1705
4,VW,Polo,1.0,Petrol,2006,127869,4101
...,...,...,...,...,...,...,...
49995,BMW,M5,5.0,Petrol,2018,28664,113006
49996,Toyota,Prius,1.8,Hybrid,2003,105120,9430
49997,Ford,Mondeo,1.6,Diesel,2022,4030,49852
49998,Ford,Focus,1.0,Diesel,2016,26468,23630


- Chuẩn hoá kiểu dữ liệu các cột

In [6]:
for c in ["brand", "model", "fuel_type"]:
    df[c] = df[c].astype(str).str.strip()

df["engine_size"]   = pd.to_numeric(df["engine_size"], errors="coerce")
df["year"]       = pd.to_numeric(df["year"], errors="coerce").astype("Int64")
df["mileage_km"] = pd.to_numeric(df["mileage_km"], errors="coerce")
df["price_usd"]  = pd.to_numeric(df["price_usd"], errors="coerce")

- Loại bỏ các dữ liệu lỗi

In [7]:
df = df[(df["price_usd"] > 0) & (df["mileage_km"] >= 0) & df["year"].notna()]

- Thêm cột age (tuổi đời của xe) và price_per_1k_km (giá/1000 km)

In [8]:
df["age"] = (2025 - df["year"].astype(int)).clip(lower=0)
df["price_per_1k_km"] = np.where(df["mileage_km"] > 0, df["price_usd"]/(df["mileage_km"]/1000), np.nan)

- thêm mileage_bucket (vùng miles mà xe đã đi được)

In [9]:
def bucket_mileage(x):
    if pd.isna(x): return np.nan
    if x < 20000:   return "<20k"
    if x < 50000:   return "20–50k"
    if x < 100000:  return "50–100k"
    return ">100k"
df["mileage_bucket"] = df["mileage_km"].apply(bucket_mileage)

- Thêm age_bucket (vùng tuổi đời của xe)

In [10]:
def bucket_age(a):
    if a < 3:   return "0–2y"
    if a < 6:   return "3–5y"
    if a < 11:  return "6–10y"
    return "10y+"
df["age_bucket"] = df["age"].apply(bucket_age)

- Thêm overprice_threshold và is_overpriced

In [11]:
def over_thr(s):
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    return q3 + 1.5*(q3 - q1)
thr = df.groupby(["brand","model"])["price_usd"].apply(over_thr).rename("overprice_threshold")
df = df.merge(thr.reset_index(), on=["brand","model"], how="left")
df["is_overpriced"] = df["price_usd"] > df["overprice_threshold"]

In [12]:
df.head()


,brand,model,engine_size,fuel_type,year,mileage_km,price_usd,age,price_per_1k_km,mileage_bucket,age_bucket,overprice_threshold,is_overpriced
0,Ford,Fiesta,1.0,Petrol,2002,127300,3074,23,24.147683,>100k,10y+,24653.000,False
1,Porsche,718 Cayman,4.0,Petrol,2016,57850,49704,9,859.187554,50–100k,6–10y,73239.750,False
2,Ford,Mondeo,1.6,Diesel,2014,39190,24072,11,614.238326,20–50k,10y+,43916.000,False
3,Toyota,RAV4,1.8,Hybrid,1988,210814,1705,37,8.087698,>100k,10y+,66286.875,False
4,VW,Polo,1.0,Petrol,2006,127869,4101,19,32.071886,>100k,10y+,26969.875,False


In [13]:
df.shape

(50000, 13)

In [14]:
df.to_csv('./Datasets/car_sales_data_cleaned.csv', index=False)